In [2]:
import tensorflow as tf
import numpy as np,sys,os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os as os
import nrrd as reader
import pandas as pd
import cv2 
import csv
from tqdm import tqdm
from tabulate import tabulate
from skimage import morphology
from skimage import measure
from skimage import segmentation
from skimage.segmentation import flood, flood_fill
from skimage.measure import label
from skimage.segmentation import active_contour
from skimage import data, io, img_as_ubyte,filters
from time import sleep
from IPython.display import clear_output

In [3]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, Concatenate
def unet(input_shape):
    inputs = Input(input_shape)
    
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    up6 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = Concatenate()([drop4, up6])
    conv6 = Conv2D(256, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)
    
    up7 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = Concatenate()([conv3, up7])
    conv7 = Conv2D(128, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(128, 3, activation='relu', padding='same')(conv7)
    
    up8 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = Concatenate()([conv2, up8])
    conv8 = Conv2D(64, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(64, 3, activation='relu', padding='same')(conv8)
    
    up9 = Conv2D(32, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = Concatenate()([conv1, up9])
    conv9 = Conv2D(32, 3, activation ='relu', padding='same')(merge9)
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [4]:
#Loading previously processed DCM and Mask arrays into NumPy arrays.
RawDCMArray = np.load("RAW-DCM-ARRAY.npy")
NormDCMArray = np.load("NORM-DCM-ARRAY.npy")
MSKMaskArray = np.load("DCM-SEG-MASK.npy")
FLEMaskArray = np.load("DCM-FLE-MASK.npy")

NameError: name 'history' is not defined

In [10]:
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

dicoms = (RawDCMArray - np.min(RawDCMArray)) / (np.max(RawDCMArray) - np.min(RawDCMArray))
masks = (FLEMaskArray - np.min(FLEMaskArray)) / (np.max(FLEMaskArray) - np.min(FLEMaskArray))

# Split your data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(dicoms, masks, test_size=0.2)

# Define your loss function and optimizer
loss = 'binary_crossentropy'
optimizer = Adam(learning_rate=1e-4)

# Compile your model
model = unet(dicoms[0].shape)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train your model using the fit() function
history = model.fit(x_train, y_train, batch_size=1, epochs=16, validation_data=(x_val, y_val))

Epoch 1/16
1608/1608 [==============================] - 635s 394ms/step - loss: 0.1050 - accuracy: 0.9550 - val_loss: 0.0417 - val_accuracy: 0.9840
Epoch 2/16
1608/1608 [==============================] - 649s 404ms/step - loss: 0.0405 - accuracy: 0.9858 - val_loss: 0.0310 - val_accuracy: 0.9893
Epoch 3/16
1608/1608 [==============================] - 650s 404ms/step - loss: 0.0338 - accuracy: 0.9886 - val_loss: 0.0243 - val_accuracy: 0.9904
Epoch 4/16
1608/1608 [==============================] - 656s 408ms/step - loss: 0.0278 - accuracy: 0.9901 - val_loss: 0.0484 - val_accuracy: 0.9850
Epoch 5/16
1608/1608 [==============================] - 625s 388ms/step - loss: 0.0350 - accuracy: 0.9875 - val_loss: 0.0210 - val_accuracy: 0.9918
Epoch 6/16
1608/1608 [==============================] - 621s 386ms/step - loss: 0.0283 - accuracy: 0.9898 - val_loss: 0.0378 - val_accuracy: 0.9893
Epoch 7/16
1608/1608 [==============================] - 628s 391ms/step - loss: 0.0257 - accuracy: 0.9910 - val_

In [ ]:
modelv2 = unet(dicoms[0].shape)
modelv2.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
modelv2val = modelv2.fit(x_train,y_train,batch_size=15,epochs=16,validation_data=(x_val,y_val))

In [15]:
#Save model
model.save("modelv1.h5")
modelv2.save("modelv2.h5")

In [16]:
#Save only model weights
model.save_weights("modelv1_weights.h5")
modelv2.save_weights("modelv2_weights.h5")

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
    show_trainable=False
)

In [18]:
#Custom UFATS UNET Model 2019

np.random.seed(678)
tf.random.set_seed(1234)

def tf_relu(x): return tf.nn.relu(x)
def d_tf_relu(s): return tf.cast(tf.greater(s,0),dtype=tf.float32)
def tf_softmax(x): return tf.nn.softmax(x)
def np_sigmoid(x): return 1/(1 + np.exp(-1 *x))

# --- hyper ---
num_epoch = 100
init_lr = 0.0001
batch_size = 2

# --- make class ---
class conlayer_left():
    
    def __init__(self,ker,in_c,out_c):
        self.w = tf.Variable(tf.random.normal([ker,ker,in_c,out_c],stddev=0.05))

    def feedforward(self,input,stride=1,dilate=1, output=1):
        self.input  = input
        self.layer  = tf.nn.conv2d(input,self.w,strides = [1,stride,stride,1],padding='SAME')
        self.layerA = tf_relu(self.layer)
        return self.layerA

class conlayer_right():
    
    def __init__(self,ker,in_c,out_c):
        self.w = tf.Variable(tf.random.normal([ker,ker,in_c,out_c],stddev=0.05))

    def feedforward(self,input,stride=1,dilate=1,output=1):
        self.input  = input
    
        current_shape_size = input.shape

        self.layer = tf.nn.conv2d_transpose(input,self.w,
        output_shape=[batch_size] + [int(current_shape_size[1]*2),int(current_shape_size[2]*2),int(current_shape_size[3]/2)],strides=[1, stride, stride, 1],padding='SAME')
        self.layerA = tf_relu(self.layer)
        return self.layerA




train_images = dicoms[:,:,:,0]
train_labels = masks[:,:,:,0]

train_images = (train_images - train_images.min()) / (train_images.max() - train_images.min())
train_labels = (train_labels - train_labels.min()) / (train_labels.max() - train_labels.min())






# --- make layer ---
# left
l1_1 = conlayer_left(3,1,3)
l1_2 = conlayer_left(3,3,3)
l1_3 = conlayer_left(3,3,3)

l2_1 = conlayer_left(3,3,6)
l2_2 = conlayer_left(3,6,6)
l2_3 = conlayer_left(3,6,6)

l3_1 = conlayer_left(3,6,12)
l3_2 = conlayer_left(3,12,12)
l3_3 = conlayer_left(3,12,12)

l4_1 = conlayer_left(3,12,24)
l4_2 = conlayer_left(3,24,24)
l4_3 = conlayer_left(3,24,24)

l5_1 = conlayer_left(3,24,48)
l5_2 = conlayer_left(3,48,48)
l5_3 = conlayer_left(3,48,24)

# right
l6_1 = conlayer_right(3,24,48)
l6_2 = conlayer_left(3,24,24)
l6_3 = conlayer_left(3,24,12)

l7_1 = conlayer_right(3,12,24)
l7_2 = conlayer_left(3,12,12)
l7_3 = conlayer_left(3,12,6)

l8_1 = conlayer_right(3,6,12)
l8_2 = conlayer_left(3,6,6)
l8_3 = conlayer_left(3,6,3)

l9_1 = conlayer_right(3,3,6)
l9_2 = conlayer_left(3,3,3)
l9_3 = conlayer_left(3,3,3)

l10_final = conlayer_left(3,3,1)

# ---- make graph ----

x = np.empty([0,256,256,1],dtype= np.float32)
y = np.empty([0,256,256,1],dtype= np.float32)

layer1_1 = l1_1.feedforward(x)
layer1_2 = l1_2.feedforward(layer1_1)
layer1_3 = l1_3.feedforward(layer1_2)

layer2_Input = tf.nn.max_pool(layer1_3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
layer2_1 = l2_1.feedforward(layer2_Input)
layer2_2 = l2_2.feedforward(layer2_1)
layer2_3 = l2_3.feedforward(layer2_2)

layer3_Input = tf.nn.max_pool(layer2_3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
layer3_1 = l3_1.feedforward(layer3_Input)
layer3_2 = l3_2.feedforward(layer3_1)
layer3_3 = l3_3.feedforward(layer3_2)

layer4_Input = tf.nn.max_pool(layer3_3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
layer4_1 = l4_1.feedforward(layer4_Input)
layer4_2 = l4_2.feedforward(layer4_1)
layer4_3 = l4_3.feedforward(layer4_2)

layer5_Input = tf.nn.max_pool(layer4_3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
layer5_1 = l5_1.feedforward(layer5_Input)
layer5_2 = l5_2.feedforward(layer5_1)
layer5_3 = l5_3.feedforward(layer5_2)

layer6_Input = tf.concat([layer5_3,layer5_Input],axis=3)
layer6_1 = l6_1.feedforward(layer6_Input)
layer6_2 = l6_2.feedforward(layer6_1)
layer6_3 = l6_3.feedforward(layer6_2)

layer7_Input = tf.concat([layer6_3,layer4_Input],axis=3)
layer7_1 = l7_1.feedforward(layer7_Input)
layer7_2 = l7_2.feedforward(layer7_1)
layer7_3 = l7_3.feedforward(layer7_2)

layer8_Input = tf.concat([layer7_3,layer3_Input],axis=3)
layer8_1 = l8_1.feedforward(layer8_Input)
layer8_2 = l8_2.feedforward(layer8_1)
layer8_3 = l8_3.feedforward(layer8_2)

layer9_Input = tf.concat([layer8_3,layer2_Input],axis=3)
layer9_1 = l9_1.feedforward(layer9_Input)
layer9_2 = l9_2.feedforward(layer9_1)
layer9_3 = l9_3.feedforward(layer9_2)

layer10_Input = layer9_3
layer10_1 = l10_final.feedforward(layer10_Input)

cost = tf.reduce_mean(tf.keras.losses.binary_crossentropy(train_labels,layer10_1))
auto_train = tf.train.AdamOptimizer(learning_rate=init_lr).minimize(cost)

# --- start session ---
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for iter in range(num_epoch):
        
        # train
        for current_batch_index in range(0,len(train_images),batch_size):
            current_batch = train_images[current_batch_index:current_batch_index+batch_size,:,:,:]
            current_label = train_labels[current_batch_index:current_batch_index+batch_size,:,:,:]
            sess_results = sess.run([cost,auto_train],feed_dict={x:current_batch,y:current_label})
            print(' Iter: ', iter, " Cost:  %.32f"% sess_results[0],end='\r')
        print('\n-----------------------')
        train_images,train_labels = shuffle(train_images,train_labels)

        if iter % 2 == 0:
            test_example =   train_images[:2,:,:,:]
            test_example_gt = train_labels[:2,:,:,:]
            sess_results = sess.run([layer10],feed_dict={x:test_example})

            sess_results = sess_results[0][0,:,:,:]
            test_example = test_example[0,:,:,:]
            test_example_gt = test_example_gt[0,:,:,:]

            plt.figure()
            plt.imshow(np.squeeze(test_example),cmap='gray')
            plt.axis('off')
            plt.title('Original Image')
            plt.savefig('train_change/'+str(iter)+"a_Original_Image.png")

            plt.figure()
            plt.imshow(np.squeeze(test_example_gt),cmap='gray')
            plt.axis('off')
            plt.title('Ground Truth Mask')
            plt.savefig('train_change/'+str(iter)+"b_Original_Mask.png")

            plt.figure()
            plt.imshow(np.squeeze(sess_results),cmap='gray')
            plt.axis('off')
            plt.title('Generated Mask')
            plt.savefig('train_change/'+str(iter)+"c_Generated_Mask.png")

            plt.figure()
            plt.imshow(np.multiply(np.squeeze(test_example),np.squeeze(test_example_gt)),cmap='gray')
            plt.axis('off')
            plt.title("Ground Truth Overlay")
            plt.savefig('train_change/'+str(iter)+"d_Original_Image_Overlay.png")

            plt.figure()
            plt.axis('off')
            plt.imshow(np.multiply(np.squeeze(test_example),np.squeeze(sess_results)),cmap='gray')
            plt.title("Generated Overlay")
            plt.savefig('train_change/'+str(iter)+"e_Generated_Image_Overlay.png")

            plt.close('all')

    for data_index in range(0,len(train_images),batch_size):
        current_batch = train_images[current_batch_index:current_batch_index+batch_size,:,:,:]
        current_label = train_labels[current_batch_index:current_batch_index+batch_size,:,:,:]
        sess_results = sess.run(layer10,feed_dict={x:current_batch})

        plt.figure()
        plt.imshow(np.squeeze(current_batch[0,:,:,:]),cmap='gray')
        plt.axis('off')
        plt.title(str(data_index)+"a_Original Image")
        plt.savefig('gif/'+str(data_index)+"a_Original_Image.png")

        plt.figure()
        plt.imshow(np.squeeze(current_label[0,:,:,:]),cmap='gray')
        plt.axis('off')
        plt.title(str(data_index)+"b_Original Mask")
        plt.savefig('gif/'+str(data_index)+"b_Original_Mask.png")
        
        plt.figure()
        plt.imshow(np.squeeze(sess_results[0,:,:,:]),cmap='gray')
        plt.axis('off')
        plt.title(str(data_index)+"c_Generated Mask")
        plt.savefig('gif/'+str(data_index)+"c_Generated_Mask.png")

        plt.figure()
        plt.imshow(np.multiply(np.squeeze(current_batch[0,:,:,:]),np.squeeze(current_label[0,:,:,:])),cmap='gray')
        plt.axis('off')
        plt.title(str(data_index)+"d_Original Image Overlay")
        plt.savefig('gif/'+str(data_index)+"d_Original_Image_Overlay.png")        
        plt.savefig('gif/'+str(data_index)+"d_Original_Image_Overlay.png")


        plt.figure()
        plt.imshow(np.multiply(np.squeeze(current_batch[0,:,:,:]),np.squeeze(sess_results[0,:,:,:])),cmap='gray')
        plt.axis('off')
        plt.title(str(data_index)+"e_Generated Image Overlay")
        plt.savefig('gif/'+str(data_index)+"e_Generated_Image_Overlay.png")

        plt.close('all')


# -- end code --

InvalidArgumentError: {{function_node __wrapped__Conv2DBackpropInput_device_/job:localhost/replica:0/task:0/device:CPU:0}} Conv2DCustomBackpropInput: input and out_backprop must have the same batch size. Input batch: 2, outbackprop batch: 0, batch_dim: 0 [Op:Conv2DBackpropInput]